## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-22-03-55-30 +0000,nypost,"Kim Jong Un recalls ‘good memories’ of Trump, ...",https://nypost.com/2025/09/21/world-news/kim-j...
1,2025-09-22-03-39-13 +0000,nyt,Trump Remembers Kirk as a Martyr While Attacki...,https://www.nytimes.com/2025/09/21/us/election...
2,2025-09-22-03-38-58 +0000,bbc,Super typhoon nears Philippines as thousands e...,https://www.bbc.com/news/articles/cge223y2e9eo...
3,2025-09-22-03-21-19 +0000,nypost,New dad receives 24-year-old letter penned by ...,https://nypost.com/2025/09/21/us-news/new-dad-...
4,2025-09-22-03-15-59 +0000,startribune,"Brooks Lee, Royce Lewis home runs help Twins c...",https://www.startribune.com/brooks-lee-royce-l...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2353/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,36
12,kirk,26
54,charlie,20
105,memorial,12
537,palestinian,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
10,2025-09-22-02-09-32 +0000,bbc,Trump hails Charlie Kirk as 'American hero' as...,https://www.bbc.com/news/articles/ckgee0x9p40o...,112
46,2025-09-21-22-51-23 +0000,nypost,Trump calls Charlie Kirk one of ‘America’s gre...,https://nypost.com/2025/09/21/us-news/trump-ca...,111
37,2025-09-21-23-30-00 +0000,wsj,President Trump and the upper echelon of his a...,https://www.wsj.com/us-news/charlie-kirk-funer...,109
58,2025-09-21-21-50-25 +0000,nypost,"President Trump, Elon Musk shake hands at Char...",https://nypost.com/2025/09/21/us-news/presiden...,109
17,2025-09-22-01-30-32 +0000,wapo,"Trump, allies cast Charlie Kirk as ‘martyr’ wh...",https://www.washingtonpost.com/nation/2025/09/...,96


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
10,112,2025-09-22-02-09-32 +0000,bbc,Trump hails Charlie Kirk as 'American hero' as...,https://www.bbc.com/news/articles/ckgee0x9p40o...
130,50,2025-09-21-13-36-12 +0000,nypost,"Canada, UK and Australia all recognize Palesti...",https://nypost.com/2025/09/21/world-news/canad...
48,46,2025-09-21-22-27-41 +0000,nypost,Erika Kirk forgives assassin who killed husban...,https://nypost.com/2025/09/21/us-news/erika-ki...
85,30,2025-09-21-20-00-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...
3,28,2025-09-22-03-21-19 +0000,nypost,New dad receives 24-year-old letter penned by ...,https://nypost.com/2025/09/21/us-news/new-dad-...
180,27,2025-09-21-05-01-20 +0000,bbc,"Israel will occupy more West Bank land, but re...",https://www.bbc.com/news/articles/c0ez9qxzl2jo...
37,25,2025-09-21-23-30-00 +0000,wsj,President Trump and the upper echelon of his a...,https://www.wsj.com/us-news/charlie-kirk-funer...
62,25,2025-09-21-21-22-56 +0000,latimes,Fan gives back Mike Trout's 400th career home ...,https://www.latimes.com/sports/story/2025-09-2...
118,23,2025-09-21-15-04-14 +0000,nypost,Jasmine Crockett scolds white Dems for not vot...,https://nypost.com/2025/09/21/us-news/jasmine-...
9,22,2025-09-22-02-26-46 +0000,nypost,"Innocent teen girl shot, man stabbed in separa...",https://nypost.com/2025/09/21/us-news/teen-bys...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
